In [33]:
from datetime import datetime
import gym
from gym.wrappers import Monitor
import gym_ple

In [34]:
# Create a random agent
class RandomAgent(object):
    def __init__(self, action_space):
        self.action_space = action_space

    def act(self, observation, reward, done):
        return self.action_space.sample()

In [51]:
env = gym.make('FlappyBird-v0')
#record each episode in a video file
env = Monitor(env, "./vid", video_callable=lambda episode_id: True, force=True)

print("Action space is: {}".format(env.action_space))

agent = RandomAgent(env.action_space)

episode_count = 10
reward = 0
done = False

for i in range(episode_count):
    ob = env.reset()
    episode_reward = 0
    
    while True:
        action = agent.act(ob, reward, done)
        ob, reward, done, _ = env.step(action)
        episode_reward += reward
        if done:
            print("Finished episode {}. Episode reward: {}". format(i+1, episode_reward))
            break
            

# Dump result info to disk
env.close()


Action space is: Discrete(2)
Finished episode 1. Episode reward: -5.0
Finished episode 2. Episode reward: -5.0
Finished episode 3. Episode reward: -5.0
Finished episode 4. Episode reward: -5.0
Finished episode 5. Episode reward: -5.0
Finished episode 6. Episode reward: -5.0
Finished episode 7. Episode reward: -5.0
Finished episode 8. Episode reward: -5.0
Finished episode 9. Episode reward: -5.0
Finished episode 10. Episode reward: -5.0


In [52]:
import glob
import base64
from IPython.display import HTML
import random

# obtain all videos 
videos = [v for v in glob.glob("vid/*.mp4")]

#display random episode
video_path = random.choice(videos)
video = io.open(video_path, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))